In [79]:
import glob
import pandas as pd
import warnings
import numpy as np

## Describe Dataset 

In [166]:
audio_paths=glob.glob('../data/audio/*/*.wav')

In [167]:
df=pd.DataFrame()
warnings.filterwarnings('ignore')

for path in audio_paths:
    path_split=path.split('/')
    part=path_split[-2]
    filename=path_split[-1]
    basename=filename.split('.')[0]
    dict_={'filename':filename,'partition':part,'basename':basename}
    df=df.append(dict_,True)

In [168]:
for i,basename in enumerate(df.basename):
    df_sample=df[df['basename']==basename]
    n=df_sample.shape[0]
    n_train=df_sample[df_sample['partition']=='Train'].shape[0]
    n_test=df_sample[df_sample['partition']=='Test'].shape[0]
    n_val=df_sample[df_sample['partition']=='Val'].shape[0]
    df.loc[i,'n_segments']=n
    df.loc[i,'n_train']=n_train
    df.loc[i,'n_val']=n_val
    df.loc[i,'n_test']=n_test

In [169]:
df.to_csv('../data/dataset_description.csv')

## Create new partitions

In [170]:
df_reduced=df.drop_duplicates(subset='basename',ignore_index=True).drop(columns=['filename','partition'])

In [248]:
df_reduced

,basename,n_segments,n_train,n_val,n_test
2,8AnIttmflwI,4.0,1.0,3.0,0.0
9,_G3kw9HwCqY,5.0,4.0,1.0,0.0
13,51KRxB3g7A8,4.0,1.0,2.0,1.0
14,okSmKH2k5lE,4.0,1.0,3.0,0.0
28,TNtcyfM9jak,6.0,5.0,1.0,0.0
...,...,...,...,...,...
3047,HkqPfxphZsE,1.0,0.0,0.0,1.0
3048,k7-dMNJDc-w,1.0,0.0,0.0,1.0
3049,ytSIdQkw3gc,1.0,0.0,0.0,1.0
3053,Wx3BaKp74Ig,1.0,0.0,0.0,1.0


In [236]:
list_=[]
list_names=[]
segments=0

for i,j in zip([1836,612],[6000,2000]):

    while segments!=j:
        a=df_reduced.sample(i,replace=False)
        segments=np.sum(a.n_segments.values)
    list_names=list(a.basename.values)
    list_.append(list_names)
    df_reduced=df_reduced[~df_reduced['basename'].isin(sum(list_,[]))]

# The remaining samples are set to be 'Test'

list_names=list(df_reduced.basename.values)
list_.append(list_names)

1836 6000
612 2000


In [242]:
df_new=df.drop(columns='partition')
parts=['Train','Val','Test']

df_final=pd.DataFrame()

for k,part in enumerate(parts):
    df_new.loc[df_new['basename'].isin(list_[k]),'part_new']=part

In [245]:
df_new

,filename,basename,n_segments,n_train,n_val,n_test,part_new
0,mDRMCwSD3Ms.001.wav,mDRMCwSD3Ms,2.0,0.0,2.0,0.0,Train
1,nUazOUcW_AM.002.wav,nUazOUcW_AM,3.0,2.0,1.0,0.0,Train
2,8AnIttmflwI.000.wav,8AnIttmflwI,4.0,1.0,3.0,0.0,Test
3,XB81sertsYU.004.wav,XB81sertsYU,2.0,1.0,1.0,0.0,Train
4,alucqviYJFE.005.wav,alucqviYJFE,6.0,4.0,2.0,0.0,Train
...,...,...,...,...,...,...,...
9995,SkNO4x-LSgE.001.wav,SkNO4x-LSgE,5.0,2.0,1.0,2.0,Train
9996,QdafN9Y_j-Y.003.wav,QdafN9Y_j-Y,4.0,1.0,2.0,1.0,Train
9997,JJs_HTelie4.000.wav,JJs_HTelie4,2.0,1.0,0.0,1.0,Train
9998,cWvZnl5XjwI.002.wav,cWvZnl5XjwI,3.0,0.0,0.0,3.0,Train


In [247]:
df_new.to_csv('../data/dataset_new_partitions.csv')